In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from geopy.geocoders import Nominatim 
from geopy import distance
import folium

In [2]:
def geocode(adress):
    geocoder = Nominatim(user_agent="Antoine")
    location = geocoder.geocode(adress)
    if location:
        coords = [location.latitude, location.longitude]
        return coords        
    else:
        None


In [8]:
# 'cotes-darmor' or 'morbihan' or 'finsitere' or 'ille-et-vilaine'
departement = input("""Dans quel département souhaites tu surfer ? \n""").lower()

Dans quel département souhaites tu surfer ? 
Morbihan


In [18]:
jour_surf = input("""Quel jour souhaites tu surfer (format aaaa-mm-dd) ?\nPour plusieurs journées merci de ne rien indiquer. \n""")

Quel jour souhaites tu surfer (format aaaa-mm-dd) ?
Pour plusieurs journées merci de ne rien indiquer. 



In [22]:
heure_surf = input("""A quelle heure souhaites tu surfer (6-9-12-15-18-21) ?\nPour plusieurs plages horaires merci de ne rien indiquer. \n""")

A quelle heure souhaites tu surfer (6-9-12-15-18-21) ?
Pour plusieurs plages horaires merci de ne rien indiquer. 
9


In [23]:
fichier_lat_long_par_spot = pd.read_csv('lat_long_spots.csv', index_col=False)

response = requests.get("https://www.surf-sentinel.com/spots-de-surf/france/liste-spots-de-surf-" + departement)
soup = BeautifulSoup(response.content, "html.parser")

communes = []
spots = []

for i in soup.find_all("div", class_="location-content") : 
    url = str(i.find("div").find("a"))
    commune_avec_slash = str(re.search(departement + "/[\w-]+", url).group(0)).replace(departement,'')
    spot_url = str(re.search(departement + "/[\w-]+/[\w-]+", url).group(0)).replace(commune_avec_slash,'').replace(departement,'').replace("/","")
    commune_url = commune_avec_slash.replace('/','')
    
    communes.append(commune_url)
    
    if spot_url == '' :
        spots.append(commune_url)
    else :
        spots.append(spot_url)

spots_main_data = pd.DataFrame()
spots_main_data['commune'] = communes
spots_main_data['spot'] = spots
spots_main_data['full_name'] = spots_main_data['commune'] + ' ' + spots_main_data['spot']
spots_main_data['url'] = 'https://www.surf-sentinel.com/surf-report/france/' + departement + '/' + spots_main_data['commune'] + '/' + spots_main_data['spot']


url = []
date = []
hour = []
water_quality = []
conditions = []
level = []
hauteur = []
note = []

for spot_url in spots_main_data['url'] : 
    meteo_spot_web_page = requests.get(spot_url)
    soup_url = BeautifulSoup(meteo_spot_web_page.content, "html.parser")
    
    for i in soup_url.find_all("div", class_="detailed-report-box hidden gocenter") :
        datetime = str(re.search("box-\d+-\d+-\d+-\d+", str(i)).group(0)).replace('box-','')
        url.append(spot_url)
        date.append(datetime[0:10])
        hour.append(datetime[11:].replace('0',''))
        
        water_quality_desc = str(re.search("strong>\w<span", str(i)).group(0)).replace('strong>','').replace('<span','')
        water_quality.append(water_quality_desc)
        level_desc = str(re.search("""metric">\d</span""", str(i)).group(0)).replace('metric">','').replace('</span','')  
        level.append(level_desc)
        
        if water_quality_desc == 'A' and level_desc == '3' :
             note.append(1)
        elif water_quality_desc == 'A' and level_desc == '2' :
             note.append(2)
        elif water_quality_desc == 'B' and level_desc == '3' :
             note.append(3)
        elif water_quality_desc == 'B' and level_desc == '2' :
             note.append(4)
        elif water_quality_desc == 'C' and level_desc == '3' :
             note.append(5)
        elif water_quality_desc == 'C' and level_desc == '2' :
             note.append(6)
        elif water_quality_desc == 'A' and level_desc == '1' :
             note.append(7)
        elif water_quality_desc == 'B' and level_desc == '1' :
             note.append(8)
        elif water_quality_desc == 'C' and level_desc == '1' :
             note.append(9)
        else :
             note.append(10)   
    
    for i in soup_url.find_all("p", class_="fl-caps previ-desc") :
        try :
            conditions = str(re.search("conditions [a-z]+", str(i)).group(0)).replace('conditions ','')
        except : 
            hauteur = str(re.search("\d.\d m en", str(i)).group(0)).replace(' m en','')     

spot_attributes = pd.DataFrame()
spot_attributes['url'] = url
spot_attributes['date'] = date
spot_attributes['hour'] = hour
spot_attributes['water_quality'] = water_quality
spot_attributes['conditions'] = conditions
spot_attributes['level'] = level
spot_attributes['hauteur'] = hauteur
spot_attributes['note'] = note

spots_previsions = pd.merge(left=spots_main_data, right=spot_attributes, on = 'url', how='left')
spots_previsions = pd.merge(left=spots_previsions, right=fichier_lat_long_par_spot, on = 'full_name', how='left')

if jour_surf :
    if heure_surf :
        display(spots_previsions[(spots_previsions.date == jour_surf)][(spots_previsions.hour == heure_surf)].sort_values(by=['note']).head(10))
    else : 
        display(spots_previsions[(spots_previsions.date == jour_surf)].sort_values(by=['note']).head(10))
else : 
    if heure_surf :
        display(spots_previsions[(spots_previsions.hour == heure_surf)].sort_values(by=['note']).head(10))
    else : 
        display(spots_previsions.sort_values(by=['note']).head(10))
    

,commune,spot,full_name,url,date,hour,water_quality,conditions,level,hauteur,note,long,lat
583,bangor,donnant,bangor donnant,https://www.surf-sentinel.com/surf-report/fran...,2023-08-15,9,A,moyennes,2,0.6,2,47.319644,-3.229840
145,plouhinec,kervegant,plouhinec kervegant,https://www.surf-sentinel.com/surf-report/fran...,2023-08-12,9,C,moyennes,3,0.6,5,47.664549,-3.225693
151,plouhinec,kervegant,plouhinec kervegant,https://www.surf-sentinel.com/surf-report/fran...,2023-08-13,9,C,moyennes,3,0.6,5,47.664549,-3.225693
697,plouharnel,ty-hoche,plouharnel ty-hoche,https://www.surf-sentinel.com/surf-report/fran...,2023-08-13,9,C,moyennes,3,0.6,5,47.596983,-3.146389
691,plouharnel,ty-hoche,plouharnel ty-hoche,https://www.surf-sentinel.com/surf-report/fran...,2023-08-12,9,C,moyennes,3,0.6,5,47.596983,-3.146389
649,saint-pierre-quiberon,port-blanc,saint-pierre-quiberon port-blanc,https://www.surf-sentinel.com/surf-report/fran...,2023-08-12,9,C,moyennes,3,0.6,5,47.520556,-3.154272
193,gavres,gavres,gavres gavres,https://www.surf-sentinel.com/surf-report/fran...,2023-08-13,9,C,moyennes,3,0.6,5,47.696641,-3.352937
607,saint-pierre-quiberon,port-bara,saint-pierre-quiberon port-bara,https://www.surf-sentinel.com/surf-report/fran...,2023-08-12,9,C,moyennes,3,0.6,5,47.514195,-3.151623
655,saint-pierre-quiberon,port-blanc,saint-pierre-quiberon port-blanc,https://www.surf-sentinel.com/surf-report/fran...,2023-08-13,9,C,moyennes,3,0.6,5,47.520556,-3.154272
235,larmor-plage,toulhars,larmor-plage toulhars,https://www.surf-sentinel.com/surf-report/fran...,2023-08-13,9,C,moyennes,3,0.6,5,47.705856,-3.385563


In [ ]:
dd

In [ ]:
geocode('guidel la-falaise')

In [ ]:
start_address = input('Where we start ? \n')

In [ ]:
spots_previsions['distance'] = round(distance.distance(geocode(start_address), geocode(spots_previsions['commune'] + ' ' + spots_previsions['spot'])).km,2)

In [ ]:
dd

In [ ]:
start_address = input('Where we start ? \n')
end_address = geocode('bangor donnant')
distance_two_points = round(distance.distance(geocode(start_address), end_address).km,2)
distance_two_points

In [ ]:
def geocode(adress):
    geocoder = Nominatim(user_agent="Antoine")
    location = geocoder.geocode(adress)
    if location:
        coords = [location.latitude, location.longitude]
        return coords
#         print(f'Result found for {adress}')
#         print(coords)
        
    else:
        None

def draw_100km_map(adress):
    coords = geocode(adress)
    if coords:
        map_ = folium.Map(location = coords, zoom_start = 10)
        folium.Marker(location = coords, tooltip = "Visit the spot", popup = "L'eau est bonne").add_to(map_)
        folium.Circle(location = coords, radius = 100 * 1000, fill=True).add_to(map_)
        return map_
    else:
        print(f"location not found for {adress}")
        return None

user_adress = input("Enter an adress: ")
map_ = draw_100km_map(adress = user_adress)
map_


In [ ]:
distance(coords_start, coords_end)

In [ ]:
def geocode(adress):
    geocoder = Nominatim(user_agent="Antoine")
    location = geocoder.geocode(adress)
    if location:
        coords = [location.latitude, location.longitude]
        return coords        
    else:
        None


In [ ]:
start_address = input('Where we start ? \n')
end_address = geocode('bangor donnant')
distance_two_points = round(distance.distance(geocode(start_address), end_address).km,2)
distance_two_points


In [ ]:
dd

In [ ]:
date

In [ ]:
hour

In [ ]:
meteo_spot_web_page = requests.get(spots_main_data['url'][0])
soup_url = BeautifulSoup(meteo_spot_web_page.content, "html.parser")

In [ ]:
n = 0
for i in soup_url.find_all("div", class_="detailed-report-box hidden gocenter") :
    datetime = str(re.search("box-\d+-\d+-\d+-\d+", str(i)).group(0)).replace('box-','')
    date = datetime[0:10]
    hour = datetime[11:].replace('0','')
    n += 1
    print('date : ', date, 'hour : ', hour, 'n', n)
    

In [ ]:
for i in soup_url.find_all("span", class_="note") :
    water_quality = str(re.search("strong>\w<span", str(i)).group(0)).replace('strong>','').replace('<span','')
    level = str(re.search("""metric">\d</span""", str(i)).group(0)).replace('metric">','').replace('</span','')  
    print(level)
    

In [ ]:
for i in soup_url.find_all("p", class_="fl-caps previ-desc") :
    try :
        conditions = str(re.search("conditions [a-z]+", str(i)).group(0)).replace('conditions ','')
        print(conditions)
    except : 
        hauteur = str(re.search("\d.\d m en", str(i)).group(0)).replace(' m en','')
#     print(conditions)
#     print(conditions)
        print(hauteur)